In [ ]:
"""
This notebook is used to find the white card in the live bee images and rotate and crop to it.
It does this by:
 1. Doing canny edge detection on the image to find the edges of the card
 2. Rotating the card so the edges are now vertical
 3. Cropping the image to the card
 4. Saving the new image with just the card
"""

In [1]:
import os 
import cv2
import time
import json
import shutil
import easyocr
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage


from copy import deepcopy
from glob import glob
from tqdm import tqdm
from skimage.feature import match_template
from scipy import ndimage
from skimage.io import imread
from scipy.signal import fftconvolve
from skimage.feature import peak_local_max

from skimage import feature

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny

from skimage.exposure import match_histograms



In [2]:
# rename some bee images (Can see the number in the image)
renames = {'../WWBEE24_images/Round01/Hive03/2024_06_20/IMG_0016.JPG':'../WWBEE24_images/Round01/Hive03/2024_06_20/h03b16.JPG',
          '../WWBEE24_images/Round02/hive15/2024_06_21/IMG_0036.JPG':'../WWBEE24_images/Round02/hive15/2024_06_21/h15b18.JPG'}
for k,v in renames.items():
    if os.path.exists(k):
        shutil.move(k,v)

In [3]:

def get_horizontal_edge_intersection_points(image, line):
    angle, dist = line
    # get where line crosses top edge
    top = int(dist/np.cos(-angle))


    # get where line crosses bottom edge
    bott = int(top + image.shape[1]*np.tan(-angle))
    return top, bott

In [4]:
DEBUG = False

# set amount to down-sample image when doing convolultions to make process faster
scale_factor = 10

# Use reference to histogram match images, makes thresholding easier
reference = cv2.imread('../WWBEE24_images/Round01/Hive02/2024_06_18/h02b16.JPG')

# Get List of Img File Paths
img_fps = glob('../WWBEE24_images/Round*/*/*/*')

img_fps = sorted(img_fps)
#np.random.shuffle(img_fps)

for img_fp in tqdm(img_fps):
    
    if DEBUG:
        img_fp = glob('../WWBEE24_images/Round*/*/2024_06_05/h01bee45.JPG')[0]


    # extract metadata from filepath
    date = img_fp.split('/')[-2]
    bee_id = img_fp.split('/')[-1].split('.')[0].replace('_','-')
    
    new_fp = '../2_live_bees/1_cards/' + date + '_' + bee_id + '.png'
    metadata_fp = '../2_live_bees/1_metadata/' + date + '_' + bee_id + '.json'
    if os.path.exists(new_fp) and os.path.exists(metadata_fp) and (not DEBUG):
        continue
    
    metadata = {}
    img = cv2.imread(img_fp)

    matched = match_histograms(img, reference, channel_axis=-1)


    
    # Scale image so it's faster to process and convert to gray so use intensity values
    img_gray = cv2.cvtColor(matched, cv2.COLOR_BGR2GRAY)
    scaled_img = img_gray[::scale_factor,::scale_factor] # crude way to scale images - just take every nth pixel
    #scaled_img = cv2.blur(scaled_img, (9,9)) # Could also blur, but this is effecively done by the edge detector anyways
    
    if DEBUG:
        plt.figure()
        plt.imshow(scaled_img)

    # Create a white rectangle kind of the same shape as the white paper background
    structured_element = (np.ones((300,120))*150).astype('uint8')

    # convolve structured element with scaled image, to find paper in images
    convolved = fftconvolve(scaled_img, structured_element, mode='same', axes=None)
    points = peak_local_max(convolved, min_distance=50, threshold_abs=10000, num_peaks=24)
    if '2024_06_06/h05bee07' in img_fp:
        points += [[200,320]]

    
    if DEBUG:
        plt.figure()
        plt.imshow(convolved)
        # visualize the peaks that we found
        for i in range(len(points)):
            convolved[points[i][0]-10:points[i][0]+10, points[i][1]-10:points[i][1]+10] = 0
        plt.figure()
        plt.imshow(convolved)
    
    # Find the peak closest to the center (usually the white paper) 
    min_dist = np.inf
    img_center = (scaled_img.shape[0]//2, scaled_img.shape[1]//2)
    for p in points:
        dist_to_center = ((p - img_center)**2).mean()
        if dist_to_center < min_dist:
            min_dist = dist_to_center
            paper_center = p*scale_factor

    if '2024_06_05/h01bee45.JPG' in img_fp:
        paper_center = [1980, 4410]
    
    # get a rectangle around the paper center
    rx, ry = 1300, 4000
    paper_sx, paper_ex = max(paper_center[1]-rx,0), min(paper_center[1]+rx, img.shape[1])

    # crop around the paper
    crop = img[:, paper_sx:paper_ex]
    
    if DEBUG:
        plt.figure()
        plt.imshow((crop < 220).astype('uint8')*255)

    scaled_crop = crop[::scale_factor,::scale_factor] 
    
    # Compute the Canny filter
    edges = feature.canny(cv2.cvtColor(scaled_crop, cv2.COLOR_BGR2GRAY), sigma=4)
    
    if DEBUG:
        plt.figure()
        plt.imshow(edges)
    
    # Classic straight-line Hough transform
    tested_angles = np.linspace(-np.pi / 4, np.pi / 4, 360, endpoint=False)
    h, theta, d = hough_line(edges, theta=tested_angles)    
    peaks = hough_line_peaks(h, theta, d, num_peaks=2)


    if DEBUG:
        # Plot detected hough lines
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        ax = axes.ravel()
        
        ax[0].imshow(edges)
        ax[0].set_title('Input image')
        ax[0].set_axis_off()
        
        ax[1].imshow(edges)
        ax[1].set_title('Detected lines')
        
        for _, angle, dist in zip(*peaks):
            (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
            ax[1].axline((x0, y0), slope=np.tan(angle + np.pi / 2))
    
    # hough line gives us the equation for a line as a distance from the origin
    # and the angle of a line going through the origin normal to the line of interest
    # We need to translate this into pixel locations at the top and bottom edges of the image (x1 and x2)
    # for both lines 
    x1,x2 = get_horizontal_edge_intersection_points(edges, (peaks[1][0], peaks[2][0]))
    # if can only find one line, use proximity to the paper center to decide if it is 
    # the left or right edge of the paper and just guess size of the paper
    if len(peaks[1]) == 1:
        if x1 > (paper_center[1] - paper_sx) // scale_factor:
            x3, x4 = x1 - 160, x2 - 160
        else:
            x3, x4 = x1 + 160, x2 + 160
    else:
        x3,x4 = get_horizontal_edge_intersection_points(edges, (peaks[1][1], peaks[2][1]))

    # find the minimum and maximum location of the paper in the image
    min_x = max(min([x1,x2,x3,x4]),0)
    max_x = min(max([x1,x2,x3,x4]),scaled_crop.shape[1])

    # get the rotation angle from one of the hough lines
    rotation_angle = peaks[1][0]

    # rotate the histogram matched image that amount
    rotated = ndimage.rotate(matched, rotation_angle*180/np.pi, mode='constant', cval=255)

    # calculate the location of min_x and max_y in the new rotated space
    rotation_x_shift = np.abs(img.shape[0]*np.sin(-rotation_angle)     )
    x_start = int(rotation_x_shift + np.cos(rotation_angle)*(paper_sx + min_x*scale_factor))
    x_end = int(rotation_x_shift + paper_sx*np.cos(-rotation_angle) + np.cos(-rotation_angle)*min(sorted([x1,x2,x3,x4])[-2],scaled_crop.shape[1])*scale_factor)


    # sometimes when we used just one peak, there is still a border around the paper, we can find and crop that using 
    # the intensities of the columns
    
    
    rotated_crop = rotated[:,x_start:x_end]
    if DEBUG:
        plt.figure()
        plt.imshow(rotated)
        plt.title('rotated')
        plt.figure()
        plt.imshow(rotated_crop)


    x_end_hist = rotated_crop.shape[1] - 1
    x_start_hist = 1

    pixels_below_thres = np.where(rotated_crop.mean(axis=2).mean(axis=0) < 150)[0]
    i = 0
    while np.in1d(x_start_hist, pixels_below_thres):
        x_start_hist += 1
        i += 1
        if i >= 100:
            break
    i = 0
    while np.in1d(x_end_hist, pixels_below_thres):
        x_end_hist -= 1
        i += 1
        if i >= 100:
            break

    
    if DEBUG:
        plt.imshow(rotated_crop)
        plt.figure()
        plt.hist(rotated_crop.mean(axis=2).mean(axis=0), bins=100)
        plt.figure()
        plt.plot(rotated_crop.mean(axis=2).mean(axis=0))
        plt.figure()
        plt.imshow(rotated_crop[:,x_start_hist:x_end_hist])
    rotated_crop = rotated[:,x_start + x_start_hist:x_start + x_end_hist]
    
    rotated = ndimage.rotate(img, rotation_angle*180/np.pi, mode='constant', cval=255)

    # Save cropped image
    cv2.imwrite(new_fp, rotated[:,x_start + x_start_hist:x_start + x_end_hist])

    # Save metadata
    metadata['card_angle'] = rotation_angle*180/np.pi
    metadata['card_rotated_x_start'] = x_start + x_start_hist
    metadata['card_rotated_x_end'] = x_start + x_end_hist
    with open(metadata_fp, "w") as outfile: 
        json.dump(metadata, outfile)
    if DEBUG:
        break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1194/1194 [1:03:59<00:00,  3.22s/it]


In [ ]:
scale